# Update

> Update functions and classes

In [ ]:
#| default_exp update

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from emb_opt.imports import *
from emb_opt.core import Module
from emb_opt.schemas import Item, Query, Batch, UpdateFunction

/home/dmai/miniconda3/envs/emb_opt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class UpdateModule(Module):
    def __init__(self,
                 function: UpdateFunction,
                ):
        super().__init__(Query, function)
        
#     def gather_inputs(self, batch: Batch) -> (List[Tuple], List[Query]):
#         idxs, inputs = batch.flatten_queries()
#         return (idxs, inputs)
    
#     def scatter_results(self, batch: Batch, idxs: List[Tuple], results: List[Query]):
#         for (q_idx, r_idx), result in zip(idxs, results):
#             batch_item = batch.get_item(q_idx, r_idx)
#             if result.data:
#                 batch_item.data.update(result.data)

#             if not result.valid:
#                 batch_item.data['_internal']['remove'] = True
#                 batch_item.data['_internal']['remove_details'] = 'prune result invalid'
                
                
        
#     def __call__(self, batch: Batch) -> Batch:
        
#         idxs, inputs = self.gather_inputs(batch)
#         results = self.function(inputs)
#         results = self.validate_schema(results)
#         self.scatter_results(batch, idxs, results)
#         return batch

how to manage collection propagation?
    in function puts burden on user
    in module requires nested result data structure